In [3]:
import psycopg
import pandas as pd
import credentials
from IPython.display import HTML

In [2]:
conn = psycopg.connect(
        host="sculptor.stat.cmu.edu", dbname=credentials.DB_USER,
        user=credentials.DB_USER, password=credentials.DB_PASSWORD)

cur = conn.cursor()

In [4]:
most_increases = pd.read_sql_query("select state, cast((covid_numbers - prev_week) AS INTEGER) as change_in_cases from "
"(select geographic_info.state, hospital_capacity.date, (sum(hospital_capacity.inpatient_beds_used_covid_7_day_avg) + sum(hospital_capacity.staffed_icu_adult_patients_confirmed_covid_7_day_avg))*7 as covid_numbers, lag((sum(hospital_capacity.inpatient_beds_used_covid_7_day_avg) + sum(hospital_capacity.staffed_icu_adult_patients_confirmed_covid_7_day_avg))*7, 1) over (PARTITION BY geographic_info.state ORDER BY hospital_capacity.date) as prev_week, "
"rank() over (PARTITION BY geographic_info.state ORDER BY hospital_capacity.date DESC) as rnk "
" from hospital_capacity inner join geographic_info on hospital_capacity.hospital_pk = geographic_info.hospital_pk"
" group by geographic_info.state, hospital_capacity.date) as df where rnk = 1 and (covid_numbers - prev_week) > 0 ORDER BY change_in_cases DESC", conn).reset_index().drop(['index'], axis=1).set_index(['state'])

/Users/nikitatcherevik/opt/miniconda3/envs/stat_computing/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [5]:
HTML(most_increases.to_html(classes='table table-stripped'))

,change_in_cases
state,
NY,1982
MI,655
AZ,451
OH,377
NJ,360
MA,337
NM,230
HI,220
NH,203
